In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [156]:
def import_annot_to_pandas(vcf_file, sep='\t'):
    """
    Order several annoattion by:
    Putative impact: Effects having higher putative impact are first.
    Effect type: Effects assumed to be more deleterious effects first.
    Canonical transcript before non-canonical.
    Marker genomic coordinates (e.g. genes starting before first)
    https://pcingola.github.io/SnpEff/se_inputoutput/
    Parse vcf outputted by snpEFF which adds the ANN field
    Dependences: calculate_ALT_AD
                calculate_true_ALT
    """
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
        
    #Use first line as header
    df = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)

    ann_headers = ['Allele',
                    'Annotation',
                    'Annotation_Impact',
                    'Gene_Name',
                    'Gene_ID',
                    'Feature_Type',
                    'Feature_ID',
                    'Transcript_BioType',
                    'Rank',
                    'HGVS.c',
                    'HGVS.p',
                    'cDNA.pos / cDNA.length',
                    'CDS.pos / CDS.length',
                    'AA.pos / AA.length',
                    'ERRORS / WARNINGS / INFO']
    anlelle_headers = ['Codon_change', 'AA_change', 'DP', 'Allele']

    #Apply function to split and recover the first 15 fields = only first anotations, the most likely

    df['TMP_ANN_16'] = df['INFO'].apply(lambda x: ('|').join(x.split('|')[0:15]))
    df[ann_headers] = df['TMP_ANN_16'].str.split('|', expand=True)
    df['HGVS.c'] = df['HGVS.c'].str.split(".").str[-1]
    df['HGVS.p'] = df['HGVS.p'].str.split(".").str[-1]
    df[anlelle_headers] = df['Allele'].str.split(';', expand=True)

    #df['Allele'] = df['Allele'].str.split("=").str[-1]
    del df['TMP_ANN_16']

    #Send INFO column to last position
    df = df[ [ col for col in df.columns if col != 'INFO' ] + ['INFO'] ]

    return df

In [89]:
df = import_annot_to_pandas('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Variants/20284978ciclos5_tannot.annot')

In [90]:
df.columns

Index([&#39;#CHROM&#39;, &#39;POS&#39;, &#39;ID&#39;, &#39;REF&#39;, &#39;ALT&#39;, &#39;QUAL&#39;, &#39;FILTER&#39;, &#39;Allele&#39;,
       &#39;Annotation&#39;, &#39;Annotation_Impact&#39;, &#39;Gene_Name&#39;, &#39;Gene_ID&#39;,
       &#39;Feature_Type&#39;, &#39;Feature_ID&#39;, &#39;Transcript_BioType&#39;, &#39;Rank&#39;, &#39;HGVS.c&#39;,
       &#39;HGVS.p&#39;, &#39;cDNA.pos / cDNA.length&#39;, &#39;CDS.pos / CDS.length&#39;,
       &#39;AA.pos / AA.length&#39;, &#39;ERRORS / WARNINGS / INFO&#39;, &#39;INFO&#39;],
      dtype=&#39;object&#39;)

In [93]:
df.to_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Variants/20284978ciclos5_tannot.annot.tsv', sep="\t")

In [13]:
ann_headers = 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO'.split('|')

In [15]:
ann_headers = [x.strip() for x in  ann_headers]

In [16]:
ann_headers

[&#39;Allele&#39;,
 &#39;Annotation&#39;,
 &#39;Annotation_Impact&#39;,
 &#39;Gene_Name&#39;,
 &#39;Gene_ID&#39;,
 &#39;Feature_Type&#39;,
 &#39;Feature_ID&#39;,
 &#39;Transcript_BioType&#39;,
 &#39;Rank&#39;,
 &#39;HGVS.c&#39;,
 &#39;HGVS.p&#39;,
 &#39;cDNA.pos / cDNA.length&#39;,
 &#39;CDS.pos / CDS.length&#39;,
 &#39;AA.pos / AA.length&#39;,
 &#39;Distance&#39;,
 &#39;ERRORS / WARNINGS / INFO&#39;]

# TSV TO VCF 4.2

In [129]:
def tsv_to_vcf42(tsv_file):
    df = pd.read_csv(tsv_file, sep="\t")
    #df.insert(2, 'ID', '.')
    df.fillna(".", inplace=True)
    df["PASS"].replace({True: 'PASS'}, inplace=True)
    df.rename(columns={"REGION": "#CHROM", "GFF_FEATURE": "ID", "ALT_QUAL": "QUAL", "PASS": "FILTER"}, inplace=True)
    df['INFO'] = df.apply(lambda x: "CODON={}-{},AA={}-{},DP={}".format(x.REF_CODON, x.ALT_CODON, x.REF_AA, x.ALT_AA, x.TOTAL_DP), axis=1)
    df = df[['#CHROM', 'POS', 'ID', 'REF', 'ALT','QUAL', 'FILTER', 'INFO']]
    
    return df

In [110]:
tsvdf = pd.read_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Variants/ivar_filtered/20353464.tsv', sep="\t")

In [130]:
tsv_to_vcf42('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Variants/ivar_filtered/20353464.tsv')

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,NC_045512.2,241,.,C,T,72,PASS,"CODON=.-.,AA=.-.,DP=1194"
1,NC_045512.2,445,cds-YP_009724389.1,T,C,69,PASS,"CODON=GTT-GTC,AA=V-V,DP=635"
2,NC_045512.2,445,cds-YP_009725295.1,T,C,69,PASS,"CODON=GTT-GTC,AA=V-V,DP=635"
3,NC_045512.2,1684,cds-YP_009724389.1,C,T,66,PASS,"CODON=ATC-ATT,AA=I-I,DP=1090"
4,NC_045512.2,1684,cds-YP_009725295.1,C,T,66,PASS,"CODON=ATC-ATT,AA=I-I,DP=1090"
5,NC_045512.2,2398,cds-YP_009724389.1,G,A,72,PASS,"CODON=ACG-ACA,AA=T-T,DP=728"
6,NC_045512.2,2398,cds-YP_009725295.1,G,A,72,PASS,"CODON=ACG-ACA,AA=T-T,DP=728"
7,NC_045512.2,3037,cds-YP_009724389.1,C,T,73,PASS,"CODON=TTC-TTT,AA=F-F,DP=641"
8,NC_045512.2,3037,cds-YP_009725295.1,C,T,73,PASS,"CODON=TTC-TTT,AA=F-F,DP=641"
9,NC_045512.2,4608,cds-YP_009724389.1,T,C,66,PASS,"CODON=ATG-ACG,AA=M-T,DP=606"


In [ ]:
#tsvdf.insert(0, 'ID', '.')

In [157]:
import_annot_to_pandas('/home/laura/ANALYSIS/covidma/Annotation/snpeff/20089101.annot')

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,Allele,Annotation,Annotation_Impact,Gene_Name,Gene_ID,Feature_Type,Feature_ID,Transcript_BioType,Rank,HGVS.c,HGVS.p,cDNA.pos / cDNA.length,CDS.pos / CDS.length,AA.pos / AA.length,ERRORS / WARNINGS / INFO,Codon_change,AA_change,DP,INFO
0,NC_045512.2,241,.,C,T,68.0,PASS,ANN=T,upstream_gene_variant,MODIFIER,ORF1ab,GU280_gp01,transcript,GU280_gp01,protein_coding,,-25C>T,,,,,25,CODON=.-.,AA=.-.,DP=749,"CODON=.-.;AA=.-.;DP=749;ANN=T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|GU280_gp01|protein_coding||c.-25C>T|||||25|,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725297.1|protein_coding||c.-25C>T|||||25|WARNING_TRANSCRIPT_NO_STOP_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742608.1|protein_coding||c.-25C>T|||||25|WARNING_TRANSCRIPT_NO_STOP_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|GU280_gp01.2|protein_coding||c.-25C>T|||||25|,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725298.1|protein_coding||c.-565C>T|||||565|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742609.1|protein_coding||c.-565C>T|||||565|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725299.1|protein_coding||c.-2479C>T|||||2479|WARNING_TRANSCRIPT_NO_START_CODON,T|upstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742610.1|protein_coding||c.-2479C>T|||||2479|WARNING_TRANSCRIPT_NO_START_CODON,T|intergenic_region|MODIFIER|CHR_START-ORF1ab|CHR_START-GU280_gp01|intergenic_region|CHR_START-GU280_gp01|||n.241C>T||||||"
1,NC_045512.2,2939,cds-YP_009724389.1,C,T,67.0,PASS,ANN=T,missense_variant,MODERATE,ORF1ab,GU280_gp01,transcript,GU280_gp01,protein_coding,1/2,2674C>T,Pro892Ser,2674/21291,2674/21291,892/7096,,CODON=CCA-TCA,AA=P-S,DP=965,"CODON=CCA-TCA;AA=P-S;DP=965;ANN=T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|GU280_gp01|protein_coding|1/2|c.2674C>T|p.Pro892Ser|2674/21291|2674/21291|892/7096||,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|GU280_gp01.2|protein_coding|1/1|c.2674C>T|p.Pro892Ser|2674/13218|2674/13218|892/4405||,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|YP_009725299.1|protein_coding|1/1|c.220C>T|p.Pro74Ser|220/5835|220/5835|74/1944||WARNING_TRANSCRIPT_NO_START_CODON,T|missense_variant|MODERATE|ORF1ab|GU280_gp01|transcript|YP_009742610.1|protein_coding|1/1|c.220C>T|p.Pro74Ser|220/5835|220/5835|74/1944||WARNING_TRANSCRIPT_NO_START_CODON,T|downstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725297.1|protein_coding||c.*2134C>T|||||2134|WARNING_TRANSCRIPT_NO_STOP_CODON,T|downstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742608.1|protein_coding||c.*2134C>T|||||2134|WARNING_TRANSCRIPT_NO_STOP_CODON,T|downstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009725298.1|protein_coding||c.*220C>T|||||220|WARNING_TRANSCRIPT_NO_START_CODON,T|downstream_gene_variant|MODIFIER|ORF1ab|GU280_gp01|transcript|YP_009742609.1|protein_coding||c.*220C>T|||||220|WARNING_TRANSCRIPT_NO_START_CODON"
2,NC_045512.2,3037,cds-YP_009724389.1,C,T,69.0,PASS,ANN=T,synonymous_variant,LOW,ORF1ab,GU280_gp01,transcript,GU280_gp01,protein_coding,1/2,2772C>T,Phe924Phe,2772/21291,2772/21291,924/7096,,CODON=TTC-TTT,AA=F-F,DP=1010,"CODON=TTC-TTT;AA=F-F;DP=1010;ANN=T|synonymous_variant|LOW|ORF1ab|GU280_gp01|transcript|GU280_gp01|protein_coding|1/2|c.2772C>T|p.Phe924Phe|2772/21291|2772/21291|924/7096||,T|synonymous_variant|LOW|ORF1ab|GU280_gp01|transcript|GU280_gp01.2|protein_coding|1/1|c.2772C>T|p.Phe924Phe|2772/13218|2772/13218|924/4405||,T|synonymous_variant|LOW|ORF1ab|GU280_gp01|transcript|YP_009725299.1|protein_coding|1/1|c.318C>T|p.Phe106Phe|318/5835|318/5835|106/1944||WARNING_TRANSCRIPT_NO_START_CODON,T|synonymous_variant|LOW|ORF1ab|GU280_gp01|transcript|YP_009742610.1|protein_coding|1/1|c.318C>T|p.Phe106Phe|318/5835|318/5835|106/1944||WARNING_TRANSC